# Get/Generate words for search



## 1 Installing needed packages

When running on a remote JupyterLab, packages that are needed have to be explicitly installed:

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys

!{sys.executable} -m pip install pandas==2.1.4

import pandas as pd
import unicodedata

## 2 Preprocess data 


In [ ]:
names_df = pd.read_csv("../data/tables/names.csv", sep=",")

# drop merged columns
names_df.drop(
    labels=[
        "note",
        "source",
    ],
    axis=1,
    inplace=True,
)

### 2.1 Remove rows with empty greek label

In [ ]:
names_df.dropna(subset=["label:el"], how="any", inplace=True, ignore_index=True)

### 2.2 Remove diacritics
As the transcripted text contains no accents and is completely lowercase, diacritcs have to be removed from data collected on persons.

In [ ]:
columns = [
    "label:el",
    "label:el:norm",
    "Nom Sg",
    "Gen Sg",
    "Dat Sg",
    "Akk Sg",
    "Voc Sg",
    "alternatives",
]

def str_remove_diacritics(s: str) -> str:
    return "".join(
        c
        for c in unicodedata.normalize("NFKD", s)
        if unicodedata.category(c) != "Mn"
    ).lower()

# remove diacritics from all columns named in columns_to_process
for col in columns:
    names_df[col] = names_df[col].apply(
        lambda x: str_remove_diacritics(x) if pd.notnull(x) else x
    )


### 2.3 Merging column data

All cases and alternative spellings get merged into a column named 'variants' to remove identical forms

In [ ]:
# Function to merge columns
def columns_to_set(row) -> set:
    filtered_list = [e for elem in row if pd.notnull(elem) for e in elem.split(',')]
    return set(filtered_list)


columns = [
    "label:el",
    "label:el:norm",
    "Nom Sg",
    "Gen Sg",
    "Dat Sg",
    "Akk Sg",
    "Voc Sg",
    "alternatives",
]

# Applying the function to merge columns
names_df["variants"] = names_df[columns].apply(columns_to_set, axis=1)

# drop merged columns
names_df.drop(
    labels=[
        "label:el",
        "Nom Sg",
        "Gen Sg",
        "Dat Sg",
        "Akk Sg",
        "Voc Sg",
        "alternatives",
    ],
    axis=1,
    inplace=True,
)

### 2.4 Add type (future use)

In [ ]:
#names_df["type"] = "name"

### 2.5 Explode by name

In [ ]:
# add index for words
names_df["wordID"] = range(0, len(names_df))
names_df = names_df.rename(columns={'variants': 'variant'})
# explode name_df by column variant
names_df = names_df.explode("variant").reset_index()
# Adding a new column 'variantID' with unique numbers for each variant (row)
names_df["variantID"] = range(0, len(names_df))

### 2.6 Remove factgrid link to keep entity numbers only

In [ ]:
# split factgrid string on ,
# Splitting strings in the column based on comma and converting them into sets
names_df["factgrid"] = (
    names_df["factgrid"].astype(str).apply(lambda x: set(x.split(",")))
)
# Exploding the sets in the column
names_df = names_df.explode("factgrid")
# Removing the substring from all strings in the column
names_df["factgrid"] = names_df["factgrid"].str.replace(
    "https://database.factgrid.de/entity/", ""
)

## 4 Write to files

In [ ]:
# drop not needed column index (as it has already been updated) from dataframe
names_df.drop(columns=["index"], inplace=True)
# write to csv file
names_df.to_csv("../data/names.csv", index=False)